<a href="https://colab.research.google.com/github/mr-nudo/intelligent-tools/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO: Import the required libraries

In [ ]:
# TODO
import os
import numpy as np
import tensorflow as tf
import random

# Adding random seed

In [ ]:
# Set environment variables
os.environ['PYTHONHASHSEED'] = str(25)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

# Set seed values
np.random.seed(25)
tf.random.set_seed(25)
random.seed(25)

# TODO: Read and Preprocess the dataset

In [ ]:
path = 'path_to_alice.txt'
text = ""

# TODO: Load and preprocess the text


In [ ]:
print(len(text))

# TODO: Using tokenizers

In [ ]:
# TODO: Tokenize the text


In [ ]:
print(total_words)

# TODO: Feature Engineering

In [ ]:
# TODO: Create input sequences
input_sequences = []


# TODO: Pad sequences

In [ ]:
print(len(input_sequences))

# TODO: Storing features and labels

In [ ]:
# TODO: Create predictors and labels


# TODO: Split the data into training and validation sets

# TODO: Building our model

In [ ]:
# TODO: Build your model

# TODO: Model training

In [ ]:
# TODO: Train your model

# TODO: Visualising the Training and Validation Accuracies and Losses against the number of Epochs

In [ ]:
# TODO: Plotting the training and validation loss and accuracy


# TODO: Generate text

In [ ]:
# TODO: Function to generate text
def generate_text():
    # TODO

# Generate text
seed_text = "Forest is"
next_words = 10
generated_text = generate_text()
print(generated_text)